# Digital Image Processing EC9570
## Garbage Classification project
2021/E/045
2021/E/179

## 1. Imports & Setup

In [20]:
import random
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import torch

## 2. Inspecting the Dataset

In [22]:
data_dir='./raw/'

In [33]:
import os
def class_count(directory):
    image_directories=[]
    for folder in os.listdir(directory):
        sub_folder_path=os.path.join(data_dir,folder)
        image_directories.append(sub_folder_path)
        print(f"{folder} - {len(os.listdir(sub_folder_path))}")


## 3. Data Augmentation Functions

In [24]:
import numpy
import cv2
def rotate(image):

    angle=random.randint(-30,30)
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rot_matrix, (w, h))

    return rotated_image


### Flip

In [25]:
def flip(image):
    flip_code=random.choice([0,1,-1])
    flipped_image = cv2.flip(image, flip_code)
    return flipped_image

### Gaussian Blur

In [26]:
# Kernel size must be an odd number
def gaussian_blur(image):
    ksize = random.choice([3, 5, 7])
    blurred_image = cv2.GaussianBlur(image, (ksize, ksize), 0)
    return blurred_image

### Random Erasing

In [27]:
# Randomly selects a rectangle region in an image and erases its pixels.
def random_erasing(image):
    erased_image = image.copy()
    (h, w) = image.shape[:2]
    # Define a random rectangle area to erase
    erase_h = random.randint(int(h * 0.1), int(h * 0.3))
    erase_w = random.randint(int(w * 0.1), int(w * 0.3))
    x1 = random.randint(0, w - erase_w)
    y1 = random.randint(0, h - erase_h)
    # Set the area to black
    erased_image[y1:y1 + erase_h, x1:x1 + erase_w] = (0, 0, 0)
    return erased_image


## 4. Balancing the Dataset

In [28]:
import os
import cv2
import random
import shutil


source_dir = './raw/'
output_dir = './processed/'
target_size = (256, 256)

# List of available augmentation functions
augmentations = [rotate, flip, gaussian_blur, random_erasing]

# 1. Calculate class sizes and find the target number of images
class_counts = {}
for folder in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder)
    if os.path.isdir(folder_path):
        class_counts[folder] = len(os.listdir(folder_path))

if not class_counts:
    print("Source directory is empty or contains no subdirectories.")
else:
    max_images = max(class_counts.values())
    print(f"Target image count per class: {max_images}")

    # 2. Create output directories and process each class
    for class_name, current_count in class_counts.items():
        print(f"Processing class: {class_name}")

        # Create corresponding output directory
        output_class_dir = os.path.join(output_dir, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        source_class_dir = os.path.join(source_dir, class_name)
        image_files = [f for f in os.listdir(source_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Copy original images
        for filename in image_files:
            shutil.copy(os.path.join(source_class_dir, filename), os.path.join(output_class_dir, filename))

        # 3. Generate new images if needed
        num_to_generate = max_images - current_count
        print(f"Generating {num_to_generate} new images for {class_name}...")

        for i in range(num_to_generate):
            # Pick a random original image to augment
            random_image_name = random.choice(image_files)
            image_path = os.path.join(source_class_dir, random_image_name)
            image = cv2.imread(image_path)

            if image is None:
                continue

            # Apply a random number of augmentations
            num_aug_to_apply = random.randint(1, len(augmentations))
            augs_to_apply = random.sample(augmentations, num_aug_to_apply)

            augmented_image = image
            for aug_func in augs_to_apply:
                augmented_image = aug_func(augmented_image)

            # Resize the final augmented image
            augmented_image = cv2.resize(augmented_image, target_size)

            # Save the new image
            base_filename, file_ext = os.path.splitext(random_image_name)
            new_filename = f"{base_filename}_aug_{i}{file_ext}"
            save_path = os.path.join(output_class_dir, new_filename)
            cv2.imwrite(save_path, augmented_image)

    print("\nDataset balancing complete.")


Target image count per class: 594
Processing class: cardboard
Generating 191 new images for cardboard...
Processing class: glass
Generating 93 new images for glass...
Processing class: metal
Generating 184 new images for metal...
Processing class: paper
Generating 0 new images for paper...
Processing class: plastic
Generating 112 new images for plastic...
Processing class: trash
Generating 457 new images for trash...

Dataset balancing complete.


In [34]:
class_count('./processed/')

cardboard - 403
glass - 501
metal - 410
paper - 594
plastic - 482
trash - 137
